<a href="https://colab.research.google.com/github/leticiaschaves/leticiaschaves/blob/data-projects/LESSON_04_DEEP_LEARNING_ajuste_fino_vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense

vgg16 = VGG16(weights = 'imagenet', input_shape = (224, 224, 3))
vgg16.summary()

modelo = Sequential()

for l in vgg16.layers[:-3]:
  modelo.add(l)

for l in modelo.layers:
  l.trainable = False

modelo.add(Dense(120, activation = 'relu'))
modelo.add(Dense(1, activation = 'sigmoid'))

modelo.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [29]:
from keras.optimizers import RMSprop

modelo.compile(optimizer = RMSprop(learning_rate = 0.0001),
               loss = 'binary_crossentropy',
               metrics = ['accuracy'])

from keras_preprocessing.image import ImageDataGenerator

diretorio_treino = '/content/drive/MyDrive/horse-or-human/train'
diretorio_validacao = '/content/drive/MyDrive/horse-or-human/validation'

train = ImageDataGenerator(rescale=1./255).flow_from_directory(diretorio_treino,
                                       target_size= (224, 224),
                                       class_mode = 'binary',
                                       batch_size = 32)

validation = ImageDataGenerator(rescale=1./255).flow_from_directory(diretorio_validacao,
                                       target_size= (224, 224),
                                       class_mode = 'binary',
                                       batch_size = 32)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [30]:
len(train.filenames)

1027

In [31]:
from keras.callbacks import EarlyStopping

modelo.fit_generator(train,
           steps_per_epoch = len(train.filenames) / 32,
           validation_data = validation,
           verbose = 1,
           callbacks = EarlyStopping(monitor = 'val_accuracy', patience = 2),
           epochs = 10)

<ipython-input-31-6d089d0f945b>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  modelo.fit_generator(train,


Epoch 1/10
32/32 [==============================] - 11s 307ms/step - loss: 0.2510 - accuracy: 0.9133 - val_loss: 0.4332 - val_accuracy: 0.8281
Epoch 2/10
32/32 [==============================] - 10s 316ms/step - loss: 0.0360 - accuracy: 0.9912 - val_loss: 0.1642 - val_accuracy: 0.9375
Epoch 3/10
32/32 [==============================] - 10s 296ms/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 0.1862 - val_accuracy: 0.9258
Epoch 4/10
32/32 [==============================] - 9s 293ms/step - loss: 0.0117 - accuracy: 0.9942 - val_loss: 0.1098 - val_accuracy: 0.9648
Epoch 5/10
32/32 [==============================] - 11s 345ms/step - loss: 7.1245e-04 - accuracy: 1.0000 - val_loss: 0.1968 - val_accuracy: 0.9336
Epoch 6/10
32/32 [==============================] - 10s 294ms/step - loss: 0.0046 - accuracy: 0.9981 - val_loss: 0.3179 - val_accuracy: 0.9023


In [32]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

image_path = '/content/drive/MyDrive/horse-or-human/cavalo.jpg'

img = image.load_img(image_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = modelo.predict(x)

1/1 [==============================] - 0s 122ms/step


In [34]:
train.class_indices

{'horses': 0, 'humans': 1}

In [37]:
preds

array([[0.]], dtype=float32)